In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%load_ext Cython

# Lattice Boltzman Adveccion Difusion

In [82]:
%%cython 
import numpy as np
cimport numpy as np
from libc.math cimport exp, sqrt, pow, M_PI
from libc.stdio cimport FILE, fopen, fclose, fprintf
from cpython cimport array
from numpy cimport PyArray_SimpleNewFromData, NPY_DOUBLE, import_array
from libc.stdlib cimport malloc, free

# Velocidad del lattice y velocidad del sonido
cdef double C = 1.0  # Velocidad característica del lattice
cdef double Cs = C / sqrt(3)  # Velocidad de la onda sonora
cdef double Cs2 = Cs * Cs  # Velocidad de la onda sonora al cuadrado

#-------------------------------VARIABLES GLOBALES------------------------

# Parámetros de difusión y relajación
cdef double D = 0.016  # Coeficiente de difusión
cdef double tau = (D / Cs2) + 0.5;  # Tiempo de relajación
cdef double Utau = 1.0 / tau  # Inverso del tiempo de relajación
cdef double UmUtau = 1 - Utau # 1 - 1/tau

#-------------------------------CLASES--------------------------------------
cdef class LatticeBoltzman:
    cdef int Lx, Ly, t_hour, iter_per_hour, Q
    cdef double* w
    cdef int index_f 
    cdef bint IsData
    cdef int* Vx, 
    cdef int *Vy, 
    
    cdef double* f  # Arreglo en C para f
    cdef double* fnew  # Arreglo en C para fnew
    
    # cdef public np.ndarray[np.double_t, ndim=1] f
    # cdef public np.ndarray[np.double_t, ndim=1] f_new
    
   
    # Usamos `object` aquí en lugar de `np.ndarray` para evitar el error de buffer
    cdef object id
    cdef object rho_f
    cdef object Ux  
    cdef object Uy  
    cdef object rho0
    
    def __cinit__(self, int Lx, int Ly, int t_hour, 
                  int iter_per_hour, 
                  np.ndarray[np.double_t, ndim=1] id,
                  np.ndarray[np.double_t, ndim=1] rho_f,
                  np.ndarray[np.double_t, ndim=1] Ux, 
                  np.ndarray[np.double_t, ndim=1] Uy,
                  np.ndarray[np.double_t, ndim=1] rho0):
        
        self.Lx = Lx
        self.Ly = Ly
        self.t_hour = t_hour
        self.iter_per_hour = iter_per_hour
        self.index_f = 0
        self.IsData = True
        
        self.Q = 9
        self.w = <double*> malloc(self.Q * sizeof(double))
        self.Vx = <int*> malloc(self.Q * sizeof(int))
        self.Vy = <int*> malloc(self.Q * sizeof(int))
        
        self.Ux = Ux
        self.Uy = Uy
        self.id = id
        self.rho_f = rho_f
        self.rho0 = rho0
        
        # Asignación de los pesos y vectores de velocidad
        self.w[0] = 4.0 / 9
        for i in range(1, 5):
            self.w[i] = 1.0 / 9
        for i in range(5, 9):
            self.w[i] = 1.0 / 36

        self.Vx[8] = 1
        self.Vx[1] = 1
        self.Vx[5] = 1
        self.Vx[4] = 0
        self.Vx[0] = 0
        self.Vx[2] = 0
        self.Vx[7] = -1
        self.Vx[3] = -1
        self.Vx[6] = -1

        self.Vy[8] = -1
        self.Vy[1] = 0
        self.Vy[5] = 1
        self.Vy[4] = -1
        self.Vy[0] = 0
        self.Vy[2] = 1
        self.Vy[7] = -1
        self.Vy[3] = 0
        self.Vy[6] = 1

        cdef int ArraySize = self.Lx * self.Ly * self.Q
        self.f = <double *> malloc(ArraySize * sizeof(double))
        self.fnew = <double *> malloc(ArraySize * sizeof(double))
        
       

    def __dealloc__(self):
        free(self.f)
        free(self.fnew)
        free(self.w)
        free(self.Vx)
        free(self.Vy)



    cdef int n(self, int ix, int iy, int i):
        return (ix * self.Ly + iy) * self.Q + i

    cdef double rho(self, int ix, int iy, bint UseNew):
        cdef double sum = 0.0
        cdef int i, n0
        for i in range(self.Q):
            n0 = self.n(ix, iy, i)
            if UseNew:
                sum += self.fnew[n0]
            else:
                sum += self.f[n0]
        return sum


    cdef double Jx(self, int ix, int iy, bint UseNew):
        cdef double sum = 0.0
        cdef int i, n0
        for i in range(self.Q):
            n0 = self.n(ix, iy, i)
            if UseNew:
                sum += self.Vx[i] * self.fnew[n0]
            else:
                sum += self.Vx[i] * self.f[n0]
        return sum

    cdef double Jy(self, int ix, int iy, bint UseNew):
        cdef double sum = 0.0
        cdef int i, n0
        for i in range(self.Q):
            n0 = self.n(ix, iy, i)
            if UseNew:
                sum += self.Vy[i] * self.fnew[n0]
            else:
                sum += self.Vy[i] * self.f[n0]
        return sum

    cdef double feq(self, double rho0, double Ux0, double Uy0, int i):
        cdef double UdotVi = Ux0 * self.Vx[i] + Uy0 * self.Vy[i]
        cdef double U2 = Ux0 * Ux0 + Uy0 * Uy0
        return rho0 * self.w[i] * (1 + UdotVi / Cs2 + (UdotVi * UdotVi) / (2.0 * Cs2 * Cs2) - U2 / (2.0 * Cs2))

    cpdef Start(self, int t):
        cdef int ix, iy, i, n0
        cdef double rho00
        cdef int auxT = t //9604 
        for ix in range(self.Lx):
            for iy in range(self.Ly):
                index0 = ix*self.Ly + iy
                index = ix * self.Ly + iy + self.Lx * self.Ly * auxT
                rho00 = self.rho0[index0] 
                Ux0 = self.Ux[index] 
                Uy0 = self.Uy[index] 
                for i in range(self.Q):
                    n0 = self.n(ix, iy, i)
                    self.f[n0] = self.feq(rho00, Ux0, Uy0, i)


    cpdef Collision(self):
        cdef int ix, iy, i, n0
        cdef double rho0, Ux0, Uy0
        for ix in range(self.Lx):
            for iy in range(self.Ly):
                rho0 = self.rho(ix, iy, False)
                Ux0 = self.Jx(ix, iy, False) / rho0
                Uy0 = self.Jy(ix, iy, False) / rho0
                for i in range(self.Q):
                    n0 = self.n(ix, iy, i)
                    self.fnew[n0] = UmUtau * self.f[n0] + Utau * self.feq(rho0, Ux0, Uy0, i)

    cpdef ImposeFields(self, int t):
        cdef int ix, iy, i, index
        cdef int auxT = t //9604 
        cdef double rho0, Ux0, Uy0
        cdef int index_tmp = 0

        for ix in range(self.Lx):
            for iy in range(self.Ly):
                index = ix * self.Ly + iy + self.Lx * self.Ly * auxT
                Ux0 = self.Ux[index]
                Uy0 = self.Uy[index]
                rho0 = self.rho(ix, iy, True)
                indexFire = ix * self.Ly + iy
                if (indexFire) in self.id:
                    # rho0=self.rho_f[indexFire]
                    rho0=self.rho_f[0] #Corregir

                for i in range(self.Q):
                    n0 = self.n(ix, iy, i)
                    self.fnew[n0] = self.feq(rho0, Ux0, Uy0, i)

    cpdef Advection(self):
        cdef int ix, iy, i, ixnext, iynext, n0, n0next
        for ix in range(self.Lx):
            for iy in range(self.Ly):
                for i in range(self.Q):
                    ixnext = ix + self.Vx[i]
                    iynext = iy + self.Vy[i]
                    if 0 <= ixnext < self.Lx and 0 <= iynext < self.Ly:
                        n0 = self.n(ix, iy, i)
                        n0next = self.n(ixnext, iynext, i)
                        self.f[n0next] = self.fnew[n0]

    cpdef np.ndarray[np.double_t, ndim=2] Data(self):
        # Crear un array de NumPy para almacenar los resultados
        cdef np.ndarray[np.double_t, ndim=2] result = np.zeros((self.Lx, self.Ly), dtype=np.double)
        cdef double rho0
        cdef int ix, iy
        cdef int step = 1

        # Rellenar el array de resultados
        for ix in range(0, self.Lx, step):
            for iy in range(0, self.Ly, step):
                rho0 = self.rho(ix, iy, False)
                result[ix, iy] = rho0  # Asignar valores al array NumPy
        return result


Content of stdout:
_cython_magic_2c061c3033d47c68803f8e3049800c73b1db0386.c
c:\Users\cterr\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
   Creando biblioteca C:\Users\cterr\.ipython\cython\Users\cterr\.ipython\cython\_cython_magic_2c061c3033d47c68803f8e3049800c73b1db0386.cp311-win_amd64.lib y objeto C:\Users\cterr\.ipython\cython\Users\cterr\.ipython\cython\_cython_magic_2c061c3033d47c68803f8e3049800c73b1db0386.cp311-win_amd64.exp
Generando código
Generación de código finalizada

## Simulacion

In [83]:
#-------------------------------SIMULATION--------------------------------------
def simulation(t_inital, t_final, id, rho_f, Ux, Uy, rho0):

    
    
    Lx=100
    Ly=140
    t_hour=240
    iter_per_hour=9604

    
    lbm = LatticeBoltzman(Lx, Ly, t_hour, iter_per_hour, id, rho_f, Ux, Uy,rho0)
    lbm.Start(t_inital)

    # Simular por 100 pasos de tiempo
    for t in range(int(t_inital), int(t_final)):
        lbm.Collision()
        lbm.ImposeFields(t)
        lbm.Advection()

    return  lbm.Data()


## Carga de Datos

In [102]:
import pandas as pd
#-------------------------------Carga de datos--------------------------------------

# Cargar datos de velocidad
data = np.loadtxt('data/velocity.txt')
Ux = data[:, 0]
Uy = data[:, 1]
# Cargar datos de densidad de incendios
#data = np.loadtxt('/home/fire_coord.txt')
#fireId = data[:, 0].astype(int)
#rho_f = data[:, 1]

# Cargar los datos desde el archivo
data = np.loadtxt('data/fire_coord.txt')

fireId = data[:, 0].astype(int)
rho_f = data[:, 1]

# Separar los datos usando -1 como delimitador
indices = np.where(fireId == -1)[0]  # Encontrar los índices donde hay -1
subarrays = np.split(fireId, indices)
fireId = [arr[arr != -1] for arr in subarrays]

indices = np.where(rho_f == -1)[0]  # Encontrar los índices donde hay -1
subarrays = np.split(rho_f, indices)
rho_f = [arr[arr != -1] for arr in subarrays]


#Parámetros de la simulación
data = np.loadtxt('data/parameters.txt')
Lx = int(data[0])
Ly = int(data[1])

#Coordenadas estaciones
#data = ('/data/estaciones_coord.txt')
data = pd.read_csv('data/estaciones_coord.txt')

data.drop(data.tail(1).index,inplace=True)

Est_ix= np.array(data['x_cell'])
Est_iy = np.array(data['y_cell'])

# EstLin = Est_ix #*Ly+Est_iy #Corregir
EstLin = Est_ix *Ly+Est_iy

numEstaciones = len(Est_ix)

#PM_estaciones = PM_base('/data/PM_base.csv')

# Leer el archivo CSV y extraer los valores numéricos
df = pd.read_csv('data/PM_base.csv', header=None)

array = df[0].to_numpy()

# Partir el arreglo en subarreglos de longitud 14
PM_estaciones = array.reshape(-1, numEstaciones)


In [ ]:
print(EstLin)

Test = np.zeros((Lx, Ly))

# Ajuste

In [108]:
def objective(rho_sources_opt, stations, density_observed_stations, sources_1D,t,Ux,Uy,rho0):
    """
    Calcula el error entre la densidad observada y la densidad simulada solo en las celdas que tienen estaciones.

    Parameters:
    - rho_sources_opt: valores optimizados de rho para las celdas fuente.
    - stations: lista de índices de celdas con estaciones.
    - density_observed: arreglo unidimensional de densidad observada.
    Returns:
    - error: suma de los cuadrados de las diferencias en las celdas de las estaciones.
    """
    # Recalcular la densidad simulada usando los valores optimizados de rho
    simulated_stations_opt = simulation(t,t+1,sources_1D,rho_sources_opt,Ux,Uy,rho0).astype(np.double).flatten()

    # Extraer las densidades en las estaciones de la densidad observada y simulada optimizada
    simulated_stations_opt = simulated_stations_opt[stations.astype(int)].astype(np.double)

    # Calcular el error cuadrático medio entre las densidades observadas y simuladas en las estaciones
    error = np.sum((density_observed_stations - simulated_stations_opt) ** 2)
    print("error", error)
    return error

def optimize_parameters_for_cells(stations, density_observed, sources_1D, rho_sources,t,Ux,Uy, rho0):
    """Optimiza los valores de rho para las celdas fuente para minimizar el error entre densidad simulada y observada en las estaciones."""
    rho_initial = rho_sources  # Valores iniciales para las celdas fuente
    bounds = [(0.5, 4.0)] * len(sources_1D)  # Restricciones sobre los valores de las celdas fuente

    obj=objective(rho_initial.astype(np.double), stations.astype(np.double), density_observed.astype(np.double), sources_1D.astype(np.double),t,Ux.astype(np.double),Uy.astype(np.double),rho0.astype(np.double))
    print("El objetivo fue{obj}")
            
    # Ejecutar la optimización
    result = minimize(objective, rho_initial.astype(np.double), args=(stations.astype(np.double), density_observed.astype(np.double), sources_1D.astype(np.double),t,Ux.astype(np.double),Uy.astype(np.double),rho0.astype(np.double)), method='L-BFGS-B', bounds=bounds,tol=0.1,options={'iprint': 99})

    # Resultados óptimos
    rho_opt = result.x
    return rho_opt

In [110]:
from scipy.optimize import minimize

# Parámetros iniciales
iter_per_hour = 9604
timesAdjusment = np.arange(0, 9604*2, 9604/2)  # Lectura de todos los tiempos para ajuste

indexFuegoHora = fireId[0].astype(np.double)
rhoFuegoInicialHora = rho_f[0].astype(np.double)

# Densidad inicial
rho_s0 = np.zeros(Lx * Ly, dtype=np.double) + 0.001

# Primera simulación
rho_s = simulation(0, 0, indexFuegoHora, rhoFuegoInicialHora, Ux, Uy, rho_s0)
# Linealizar rho_s
rho_s_linear = rho_s.flatten().astype(np.double)

# Apertura de archivo de salida una vez antes del bucle
with open('OutputInfo.txt', 'a') as f:

    for i, t in enumerate(timesAdjusment):
        # Determinar el tiempo final
        tInicial = timesAdjusment[i]
        if i != len(timesAdjusment) - 1:  # Corregido el condicional
            tFinal = timesAdjusment[i+1]
        else:
            break

        # Cálculo para la hora actual
        estacionesHora = EstLin.astype(int)  # Convertir EstLin a un array de enteros
        indexHora = int((t + 1) // (iter_per_hour * 24))
        valoresEstacionesHora = PM_estaciones[indexHora].astype(np.double)
        indexFuegoHora = fireId[indexHora].astype(np.double)
        rhoFuegoInicialHora = rho_f[indexHora].astype(np.double)

        # Optimización de los parámetros
        rho_opt = optimize_parameters_for_cells(estacionesHora, 
                                                valoresEstacionesHora,
                                                indexFuegoHora,
                                                rhoFuegoInicialHora, t, Ux, Uy, rho_s_linear)
        
        rho_opt_linear = rho_opt.flatten().astype(np.double)

        # Concatenar los resultados y guardar
        data = np.concatenate((rho_opt_linear, indexFuegoHora))  # Agregar paréntesis a np.concatenate
        np.savetxt(f, data, delimiter=',')

        # Ejecutar la simulación con los parámetros optimizados
        rho0 = simulation(tInicial, tFinal, indexFuegoHora, rhoFuegoInicialHora, Ux, Uy, rho_s_linear)


simulated_stations_opt [0.00099999 0.00100011 0.00100038 0.00100033 0.00100023 0.00100004
 0.00099978 0.00100009 0.00100005 0.00100013 0.001      0.00100008
 0.00099995 0.001     ]
density_observed_stations [0.  5.  0.  0.  0.  0.  0.  0.  0.  2.  5.6 0.  0.  0. ]
error 60.33481234387667
El objetivo fue{obj}
simulated_stations_opt [0.00099999 0.00100011 0.00100038 0.00100033 0.00100023 0.00100004
 0.00099978 0.00100009 0.00100005 0.00100013 0.001      0.00100008
 0.00099995 0.001     ]
density_observed_stations [0.  5.  0.  0.  0.  0.  0.  0.  0.  2.  5.6 0.  0.  0. ]
error 60.33481234387667
simulated_stations_opt [0.00099999 0.00100011 0.00100038 0.00100033 0.00100023 0.00100004
 0.00099978 0.00100009 0.00100005 0.00100013 0.001      0.00100008
 0.00099995 0.001     ]
density_observed_stations [0.  5.  0.  0.  0.  0.  0.  0.  0.  2.  5.6 0.  0.  0. ]
error 60.33481234387667
simulated_stations_opt [0.00099999 0.00100011 0.00100038 0.00100033 0.00100023 0.00100004
 0.00099978 0.00100009